**APPLICATION OF DEEP LEARNING IN PREDICTING CURVES**

- Yield curve is a graph showing interest rates (bonds) having different maturities. 

**Project Goal:** To find and understand the factors that affect movements pf yield curve. 
- Among the variables are the historical data of the yield curve, federal debt percentage that is held by the investors, and the corporate bond.
- The dependent variable will be the 1-month, 5-year and 30-year tenure of the yield curve.
- Data sources are FRED and Yahoo websites.
- The project uses polynomials of various degrees to forecast the yield curves.